In [1]:
import numpy as np
import scipy.special as scp
from collections import deque
import random

In [2]:
class Task:
    
    def __init__(self, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5):
        self.T = 0
        if(np.random.uniform() < q):
            self.T = T0
        else:
            self.T = round(T0 - Ybar * np.log(np.random.uniform()))
        beta = (N * (T0 + (1 - q) * Ybar)) / 2
        EX = beta * scp.gamma(1 + 1 / alpha)
        aux = round(beta * ((-np.log(np.random.uniform())) ** (1 / alpha)))
        self.X = max([1, min([100 * EX, aux])])
        self.start_time = 0
        self.end_time = 0
        
class Server:
    
    def __init__(self):
        self.queue = deque()
        self.current_task = None
        
    def task_remaining_time(self):
        if(self.current_task is None):
            return(float("+Inf"))
        else:
            return(self.current_task.X)
        
    def add_task(self, task):
        if(self.current_task is None):
            self.current_task = task
        else:
            self.queue.append(task)
        
    def go_next_task(self):
        previous_task = self.current_task
        if(len(self.queue) == 0):
            self.current_task = None
        else:
            self.current_task = self.queue.pop()
        return(previous_task)

In [3]:
def JSQ_simulation(messages, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5):
    servers = [Server() for _ in range(N)]
    tasks = deque([Task(T0, 1, Ybar, N, alpha) for _ in range(messages)])
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            chosen_server = min(servers, key = lambda x : len(x.queue))
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
    return(times)

In [4]:
def JBT_simulation(messages, threshold = 3, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5):
    servers = [Server() for _ in range(N)]
    tasks = deque([Task(T0, 1, Ybar, N, alpha) for _ in range(messages)])
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            below_threshold = [server for server in servers if len(server.queue) <= 2]
            if(len(below_threshold) == 0):
                chosen_server = random.choice(servers)
            else:
                chosen_server = random.choice(below_threshold)
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
    return(times)

In [14]:
def POD_simulation(messages, d = 3, T0 = 1, q = 3 / 5, Ybar = 10, N = 20, alpha = 0.5):
    servers = [Server() for _ in range(N)]
    tasks = deque([Task(T0, 1, Ybar, N, alpha) for _ in range(messages)])
    times = []
    next_task = tasks.pop()
    current_time = 0
    while(not next_task is None):
        first_server_stopped = min(servers, key = lambda x : x.task_remaining_time()).task_remaining_time()
        servers_stopped = [server for server in servers if server.task_remaining_time() == first_server_stopped]
        time = min([first_server_stopped, next_task.T])
        current_time = current_time + time
        changed_task = False
        for server in servers:
            if(server.task_remaining_time() != first_server_stopped and not server.current_task is None):
                server.current_task.X = server.current_task.X - time
            elif(server.task_remaining_time() == first_server_stopped and time != first_server_stopped and first_server_stopped != float("+Inf")):
                server.current_task.X = server.current_task.X - time
        if(first_server_stopped <= next_task.T):
            for server in servers_stopped:
                completed_task = server.go_next_task()
                times.append(current_time - completed_task.start_time)
        if(next_task.T <= first_server_stopped):
            random_servers = random.choices(servers, k = d)
            chosen_server = min(random_servers, key = lambda x : len(x.queue))
            chosen_server.add_task(next_task)
            next_task.start_time = current_time
            if(len(tasks) == 0):
                next_task = None
            else:
                next_task = tasks.pop()
            changed_task = True
        if(not changed_task):
            next_task.T = next_task.T - time
    for server in servers:
        if(not server.current_task is None):
            times.append(current_time + server.current_task.X - server.current_task.start_time)
    return(times)